# Example usage of fixed grid Fourier integration

This notebook shows how `neffint.fourier_integral_fixed_sampling` can be used to calculate fourier integrals. This function is well suited for computing the fourier integrals of functions on a already defined frequency grid which captures all important features of the function to be transformed. More precisely, interpolating the function over the the given frequencies should give an interpolating polynomial which closely approximates the function itself.

#### Do imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import ArrayLike

import neffint as nft

#### Define some functions to measure error

In [ ]:
def relative_diff(x: float, y: float) -> float:
    """Relative difference"""
    if max(abs(x), abs(y)) == 0:
        return 0
    return abs(x-y)/max(abs(x), abs(y))
relative_diff = np.vectorize(relative_diff)

def absolute_diff(x: ArrayLike, y: ArrayLike):
    """Absolute difference"""
    return np.abs(x-y)

#### Define a function

Here, we choose a function where we know analytically what the fourier integral is. Of course, in a real setting, you would choose a function without an analytically computable fourier integral.

In [ ]:
def inv_sqrt(f: ArrayLike) -> ArrayLike:
    return 1 / np.sqrt(2*np.pi*f)

# The analytic fourier integral of inv_sqrt
def inv_sqrt_analytic_integral(t: ArrayLike):
    return np.sqrt(np.pi / (2 * t))


#### Define frequencies and times
Define some frequencies as a starting point for the algorithm. It is possible to do as is done here and only provide two frequencies, and still get decent results. It is however *recommended* to follow the following two guidelines when giving an initial frequency array:
1. **Make sure all important features of the function are within the endpoints of your array**. The algorithm does a simple scan to higher and lower frequencies, but it is better at adding helpful points to the interior of the initial range.
2. **Try to supply a good initial guess of frequencies** that capture the important features of the function. The algorithm starts with doing a rough scan to search for features, then gradually hones in on a finer mesh around the features it finds. However, if there is too much space between the points in the starting array, the rough scan might miss important features.

Examples of the effect of these guidelines will be showed below.

Also define the times you want to calculate the integral for.

In [ ]:
# Here we only supply 2 frequencies
# In doing so, we break with the second guideline
# We still try to follow the first guideline by having the frequencies span a huge range.
frequencies = (1e-10, 1e20)

times = np.logspace(-15, 0, 100)

#### Define interpolation error metric

A function must be provided that reduces the output interpolation error down to only the frequency dimension, and applies some error metric. For functions with 1D outputs, this will most often just be the absolute difference between the function and the interpolant. For multidimensional outputs, it might for example be the rms error at each frequency, the maximum error at each frequency, or some weighted average.

In [ ]:
interpolation_error_metric = lambda func_output, interpolant_output: np.abs(func_output - interpolant_output)

#### Use adaptive algorithm to compute fourier integral

In [ ]:
frequencies, func_arr = nft.improve_frequency_range(
    times=times,
    initial_frequencies=frequencies,
    func=inv_sqrt,
    interpolation_error_metric=interpolation_error_metric,
    absolute_integral_tolerance=1e0, # The absolute tolerance the algorithm tries to get the error below
    frequency_bound_scan_logstep=2**0.2, # The multiplicative step size used to scan for higher and lower frequencies to add
    bisection_mode_condition=None # None (the default) here gives only logarithmic bisection when adding internal points
)

transform_arr = nft.fourier_integral_fixed_sampling(
        times=times,
        frequencies=frequencies,
        func_values=func_arr,
        inf_correction_term=True,
        interpolation="pchip"
    )

# The two steps above are also combined into the function fourier_integral_adaptive, if one is not interested in the frequencies and func_values used for the Fourier integral itself.

# Also make an array of the analytically expected values, for comparison
transform_arr_analytic = inv_sqrt_analytic_integral(times)

#### Check out the intermediate results

In [ ]:
print(f"The final number of frequencies: {len(frequencies)}")

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

# To see the results on a linear scale, change the two lines below
ax1.set_xscale("log") # "log" -> "linear"
bins = np.geomspace(frequencies[0], frequencies[-1], 50) # np.geomspace -> np.linspace (with the same arguments)

ax1.hist(frequencies, bins=bins)
ax1.set_ylabel("# frequencies in refined array")

ax2.plot(frequencies[::100], inv_sqrt(frequencies[::100]), "r")
ax2.set_yscale("log")
ax2.set_ylabel("func(frequencies)")

ax1.set_xlabel("frequencies")
plt.show()

#### Plot the transform

In [ ]:
# Select component to plot, feel free to change np.real to e.g. np.imag, np.abs, or np.angle
f1 = np.real(transform_arr)
f2 = np.real(transform_arr_analytic)

# Select a difference metric, either relative_diff or absolute_diff
# With absolute_diff, you should hopefully see that all the points have less than 1e0 (tolerance) error
diff = absolute_diff 


fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,5))
ax1.plot(times, f1, "-o", markersize=4, label="neffint")
ax1.plot(times, f2, "-o", markersize=4, label="analytic")
ax1.semilogx() # Feel free to change to ax1.loglog
ax1.legend()
ax1.set_title("Fourier transform outputs")
ax1.set_xlabel("t /s")

ax2.plot(times, diff(f1, f2), "-o", markersize=4)
ax2.loglog()
ax2.set_title(diff.__doc__)
ax2.set_xlabel("t /s")

plt.tight_layout()
plt.show()

# TODO: ADD EXAMPLES OF BAD FREQUENCY INPUTS

- Too short range (too high low freq, or too low high freq)
- Too few freqs
